# 🌊 Optimisation de Routage UWSN avec PPO (Deep Reinforcement Learning)

Ce notebook présente une implémentation complète de l'optimisation de routage dans les réseaux de capteurs sous-marins (UWSN) en utilisant l'algorithme PPO (Proximal Policy Optimization).

## 📋 Table des matières
1. [Installation et Configuration](#installation)
2. [Modèles Physiques](#physiques)
3. [Environnement Gym](#environnement)
4. [Entraînement PPO](#entrainement)
5. [Évaluation et Comparaison](#evaluation)
6. [Visualisations](#visualisations)
7. [Sauvegarde et Téléchargement](#sauvegarde)

---


## 1. Installation et Configuration {#installation}


In [ ]:
# Installation des dépendances
!pip install stable-baselines3[extra] gymnasium matplotlib plotly pandas scipy seaborn tqdm tensorboard shimmy

# Vérification de l'installation
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

# Imports nécessaires
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import seaborn as sns
from typing import List, Dict, Any, Tuple
import random
import time
from dataclasses import dataclass
import os

# Configuration matplotlib
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Installation terminée avec succès!")


In [ ]:
# Définition des classes et fonctions pour UWSN
# (Code complet des modules src/utils_network.py et src/env_gym.py)

@dataclass
class Node:
    """Représente un nœud du réseau sous-marin"""
    id: int
    x: float  # Position x (m)
    y: float  # Position y (m)
    z: float  # Profondeur (m, négative)
    energy: float  # Énergie restante (J)
    temperature: float = 15.0  # Température (°C)
    salinity: float = 35.0  # Salinité (PSU)
    max_energy: float = 1000.0  # Énergie maximale (J)
    transmission_power: float = 1.0  # Puissance de transmission (W)
    frequency: float = 25.0  # Fréquence acoustique (kHz)
    
    def is_alive(self) -> bool:
        """Vérifie si le nœud a encore de l'énergie"""
        return self.energy > 0
    
    def distance_to(self, other: 'Node') -> float:
        """Calcule la distance 3D vers un autre nœud"""
        return np.sqrt((self.x - other.x)**2 + (self.y - other.y)**2 + (self.z - other.z)**2)

print("✅ Classes de base définies")


In [ ]:
# Modèle de propagation acoustique sous-marine
class AcousticPropagation:
    """Classe pour calculer la propagation acoustique sous-marine"""
    
    def __init__(self):
        self.c0 = 1500.0  # Vitesse du son dans l'eau (m/s)
        self.rho0 = 1025.0  # Densité de l'eau (kg/m³)
        
    def sound_speed(self, temperature: float, salinity: float, depth: float) -> float:
        """Calcule la vitesse du son selon l'équation de Mackenzie"""
        T = temperature
        S = salinity
        D = abs(depth)  # Profondeur positive
        
        c = (1448.96 + 4.591*T - 5.304e-2*T**2 + 2.374e-4*T**3 +
             1.340*(S-35) + 1.630e-2*D + 1.675e-7*D**2 -
             1.025e-2*T*(S-35) - 7.139e-13*T*D**3)
        
        return c
    
    def absorption_coefficient(self, frequency: float, temperature: float, 
                             salinity: float, depth: float, ph: float = 8.0) -> float:
        """Calcule le coefficient d'absorption acoustique selon Francois & Garrison"""
        f = frequency
        T = temperature
        S = salinity
        D = abs(depth)
        
        # Calcul de la pression (Pa)
        P = 1.01325e5 + 1.025e4 * D
        
        # Coefficients pour l'absorption
        A1 = 0.106 * np.exp((T - 26) / 9)
        A2 = 0.52 * (1 + T / 43) * (S / 35)
        A3 = 0.00049 * np.exp(-(T / 27 + D / 17))
        
        f1 = 0.78 * np.sqrt(S / 35) * np.exp(T / 26)
        f2 = 42 * np.exp(T / 17)
        
        P1 = 1
        P2 = 1 - 1.37e-4 * D + 6.2e-9 * D**2
        P3 = 1 - 3.84e-4 * D + 7.57e-8 * D**2
        
        # Coefficient d'absorption (dB/km)
        alpha = (A1 * P1 * f1 * f**2 / (f1**2 + f**2) +
                A2 * P2 * f2 * f**2 / (f2**2 + f**2) +
                A3 * P3 * f**2)
        
        return alpha
    
    def path_loss(self, distance: float, frequency: float, temperature: float,
                  salinity: float, depth: float) -> float:
        """Calcule la perte de trajet acoustique (dB)"""
        # Perte de trajet géométrique (dB)
        geometric_loss = 20 * np.log10(distance)
        
        # Perte d'absorption (dB)
        alpha = self.absorption_coefficient(frequency, temperature, salinity, depth)
        absorption_loss = alpha * distance / 1000
        
        return geometric_loss + absorption_loss

print("✅ Modèle de propagation acoustique défini")


In [ ]:
# Modèle de consommation énergétique
class EnergyModel:
    """Modèle de consommation énergétique pour les nœuds UWSN"""
    
    def __init__(self):
        self.elec = 50e-9  # Énergie par bit pour l'électronique (J/bit)
        self.amp = 1e-12  # Énergie d'amplification (J/bit/m²)
        self.recv = 50e-9  # Énergie de réception (J/bit)
        self.idle = 1e-6  # Énergie en veille (J/s)
        
    def transmission_energy(self, data_size: int, distance: float, 
                           transmission_power: float) -> float:
        """Calcule l'énergie de transmission (J)"""
        return (self.elec + self.amp * distance**2) * data_size
    
    def reception_energy(self, data_size: int) -> float:
        """Calcule l'énergie de réception (J)"""
        return self.elec * data_size

# Fonction pour créer un réseau de test
def create_sample_network(num_nodes: int = 10, area_size: float = 1000.0, 
                        depth_range: Tuple[float, float] = (-100, -10)) -> List[Node]:
    """Crée un réseau de test avec des nœuds aléatoirement positionnés"""
    np.random.seed(42)  # Pour la reproductibilité
    
    nodes = []
    for i in range(num_nodes):
        node = Node(
            id=i,
            x=np.random.uniform(0, area_size),
            y=np.random.uniform(0, area_size),
            z=np.random.uniform(depth_range[0], depth_range[1]),
            energy=np.random.uniform(500, 1000),
            temperature=np.random.uniform(10, 20),
            salinity=np.random.uniform(33, 37),
            max_energy=1000.0,
            transmission_power=np.random.uniform(0.5, 2.0),
            frequency=np.random.uniform(20, 30)
        )
        nodes.append(node)
    
    return nodes

print("✅ Modèle énergétique et fonctions utilitaires définis")


## 2. Modèles Physiques {#physiques}

### 🌊 Propagation Acoustique Sous-marine

Les modèles physiques utilisés sont basés sur des équations réalistes :

1. **Vitesse du son** (Équation de Mackenzie) :
   ```
   c(T,S,D) = 1448.96 + 4.591*T - 5.304×10⁻²*T² + 2.374×10⁻⁴*T³
              + 1.340*(S-35) + 1.630×10⁻²*D + 1.675×10⁻⁷*D²
              - 1.025×10⁻²*T*(S-35) - 7.139×10⁻¹³*T*D³
   ```

2. **Absorption acoustique** (Francois & Garrison) :
   ```
   α(f,T,S,D) = A₁*P₁*f₁*f²/(f₁² + f²) + A₂*P₂*f₂*f²/(f₂² + f²) + A₃*P₃*f²
   ```

3. **Perte de trajet** :
   ```
   TL = 20*log₁₀(d) + α*d/1000
   ```

### ⚡ Modèle Énergétique

- **Transmission** : E_tx = (E_elec + E_amp * d²) * k
- **Réception** : E_rx = E_elec * k
- **Veille** : E_idle = E_idle * t


In [ ]:
# Test des modèles physiques
print("🧪 Test des modèles physiques...")

# Création d'un réseau de test
nodes = create_sample_network(num_nodes=8, area_size=500.0)
print(f"Réseau créé avec {len(nodes)} nœuds")

# Test de propagation acoustique
acoustic = AcousticPropagation()
energy_model = EnergyModel()

# Exemple de calcul pour deux nœuds
node1, node2 = nodes[0], nodes[1]
distance = node1.distance_to(node2)

print(f"\n📊 Exemple de calcul entre nœuds {node1.id} et {node2.id}:")
print(f"Distance: {distance:.2f} m")

# Vitesse du son
sound_speed = acoustic.sound_speed(node1.temperature, node1.salinity, node1.z)
print(f"Vitesse du son: {sound_speed:.2f} m/s")

# Perte acoustique
path_loss = acoustic.path_loss(distance, node1.frequency, node1.temperature, 
                              node1.salinity, node1.z)
print(f"Perte acoustique: {path_loss:.2f} dB")

# Consommation énergétique
data_size = 1000  # bits
tx_energy = energy_model.transmission_energy(data_size, distance, node1.transmission_power)
rx_energy = energy_model.reception_energy(data_size)
total_energy = tx_energy + rx_energy

print(f"Énergie de transmission: {tx_energy*1e6:.2f} μJ")
print(f"Énergie de réception: {rx_energy*1e6:.2f} μJ")
print(f"Énergie totale: {total_energy*1e6:.2f} μJ")

print("\n✅ Modèles physiques validés!")


## 3. Environnement Gym {#environnement}

Création de l'environnement Gym personnalisé pour l'entraînement PPO.


In [ ]:
# Import de Gymnasium (compatible avec Stable-Baselines3)
import gymnasium as gym
from gymnasium import spaces

@dataclass
class UWSNState:
    """État du réseau UWSN"""
    current_node: int
    destination: int
    data_size: int
    nodes_energy: np.ndarray
    nodes_position: np.ndarray
    nodes_temperature: np.ndarray
    nodes_salinity: np.ndarray
    visited_nodes: List[int]
    path: List[int]

class UWSNRoutingEnv(gym.Env):
    """
    Environnement Gym pour l'optimisation de routage UWSN
    
    L'agent doit trouver le chemin optimal entre un nœud source et un nœud destination
    en minimisant la consommation énergétique et en tenant compte des contraintes acoustiques.
    """
    
    metadata = {'render.modes': ['human', 'rgb_array']}
    
    def __init__(self, nodes: List[Node], max_steps: int = 50, 
                 data_size_range: Tuple[int, int] = (500, 2000)):
        """
        Initialise l'environnement UWSN
        
        Args:
            nodes: Liste des nœuds du réseau
            max_steps: Nombre maximum d'étapes par épisode
            data_size_range: Plage de taille de données (bits)
        """
        super(UWSNRoutingEnv, self).__init__()
        
        self.nodes = nodes
        self.num_nodes = len(nodes)
        self.max_steps = max_steps
        self.data_size_range = data_size_range
        
        # Modèles physiques
        self.acoustic = AcousticPropagation()
        self.energy_model = EnergyModel()
        
        # Espaces d'observation et d'action
        self.observation_space = self._create_observation_space()
        self.action_space = spaces.Discrete(self.num_nodes)
        
        # État actuel
        self.state: Optional[UWSNState] = None
        self.step_count = 0
        self.episode_reward = 0.0
        
        # Statistiques
        self.episode_stats = {
            'total_energy': 0.0,
            'total_distance': 0.0,
            'num_hops': 0,
            'success': False,
            'path': []
        }
    
    def _create_observation_space(self) -> spaces.Box:
        """Crée l'espace d'observation"""
        # Observation: [current_node, destination, data_size, nodes_energy, 
        #               nodes_position, nodes_temperature, nodes_salinity, 
        #               visited_mask, path_length]
        obs_dim = (
            3 +  # current_node, destination, data_size (normalisés)
            self.num_nodes +  # nodes_energy
            self.num_nodes * 3 +  # nodes_position (x, y, z)
            self.num_nodes +  # nodes_temperature
            self.num_nodes +  # nodes_salinity
            self.num_nodes +  # visited_mask
            1  # path_length
        )
        
        return spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(obs_dim,),
            dtype=np.float32
        )
    
    def reset(self, seed=None, options=None):
        """Réinitialise l'environnement pour un nouvel épisode"""
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)
        
        # Sélection aléatoire de source et destination
        self.source = random.randint(0, self.num_nodes - 1)
        self.destination = random.randint(0, self.num_nodes - 1)
        
        # Éviter que source == destination
        while self.destination == self.source:
            self.destination = random.randint(0, self.num_nodes - 1)
        
        # Taille de données aléatoire
        self.data_size = random.randint(*self.data_size_range)
        
        # Initialisation de l'état
        self.state = UWSNState(
            current_node=self.source,
            destination=self.destination,
            data_size=self.data_size,
            nodes_energy=self._get_nodes_energy(),
            nodes_position=self._get_nodes_position(),
            nodes_temperature=self._get_nodes_temperature(),
            nodes_salinity=self._get_nodes_salinity(),
            visited_nodes=[self.source],
            path=[self.source]
        )
        
        self.step_count = 0
        self.episode_reward = 0.0
        
        # Réinitialisation des statistiques
        self.episode_stats = {
            'total_energy': 0.0,
            'total_distance': 0.0,
            'num_hops': 0,
            'success': False,
            'path': [self.source]
        }
        
        return self._get_observation(), {}
    
    def step(self, action: int) -> Tuple[np.ndarray, float, bool, bool, Dict[str, Any]]:
        """
        Exécute une action dans l'environnement
        
        Args:
            action: ID du nœud de destination choisi
        
        Returns:
            observation: Nouvel état observé
            reward: Récompense obtenue
            terminated: Si l'épisode est terminé (succès/échec)
            truncated: Si l'épisode est tronqué (limite de temps)
            info: Informations supplémentaires
        """
        if self.state is None:
            raise ValueError("L'environnement doit être réinitialisé avant step()")
        
        self.step_count += 1
        
        # Vérification de la validité de l'action
        if action < 0 or action >= self.num_nodes:
            return self._get_observation(), -100.0, True, False, {"error": "Action invalide"}
        
        if action == self.state.current_node:
            return self._get_observation(), -10.0, False, False, {"error": "Même nœud"}
        
        if not self.nodes[action].is_alive():
            return self._get_observation(), -50.0, False, False, {"error": "Nœud mort"}
        
        # Calcul de la récompense
        reward = self._calculate_reward(action)
        
        # Mise à jour de l'état
        self.state.current_node = action
        self.state.visited_nodes.append(action)
        self.state.path.append(action)
        
        # Vérification de la fin d'épisode
        terminated = self._is_done()
        truncated = self.step_count >= self.max_steps
        
        if terminated:
            self.episode_stats['success'] = (action == self.destination)
            self.episode_stats['path'] = self.state.path.copy()
        
        # Informations supplémentaires
        info = self._get_info()
        
        return self._get_observation(), reward, terminated, truncated, info
    
    def _calculate_reward(self, next_node: int) -> float:
        """Calcule la récompense pour l'action choisie"""
        current_node = self.state.current_node
        destination = self.state.destination
        
        # Distance et énergie
        distance = self.nodes[current_node].distance_to(self.nodes[next_node])
        
        # Énergie de transmission
        tx_energy = self.energy_model.transmission_energy(
            self.data_size, distance, self.nodes[current_node].transmission_power
        )
        
        # Énergie de réception
        rx_energy = self.energy_model.reception_energy(self.data_size)
        
        total_energy = tx_energy + rx_energy
        
        # Mise à jour des statistiques
        self.episode_stats['total_energy'] += total_energy
        self.episode_stats['total_distance'] += distance
        self.episode_stats['num_hops'] += 1
        
        # Récompense basée sur l'énergie (négative pour minimiser)
        energy_reward = -total_energy / 1000.0  # Normalisation
        
        # Récompense de distance (pénalité pour les longs trajets)
        distance_penalty = -distance / 1000.0
        
        # Récompense de succès (arrivée à destination)
        success_reward = 100.0 if next_node == destination else 0.0
        
        # Pénalité pour les nœuds déjà visités (éviter les boucles)
        visited_penalty = -5.0 if next_node in self.state.visited_nodes[:-1] else 0.0
        
        # Pénalité pour les nœuds avec peu d'énergie
        energy_penalty = -10.0 if self.nodes[next_node].energy < 200 else 0.0
        
        # Pénalité pour les étapes trop longues
        step_penalty = -1.0 if self.step_count > self.max_steps * 0.8 else 0.0
        
        # Récompense de proximité à la destination
        current_to_dest = self.nodes[current_node].distance_to(self.nodes[destination])
        next_to_dest = self.nodes[next_node].distance_to(self.nodes[destination])
        proximity_reward = (current_to_dest - next_to_dest) / 100.0
        
        total_reward = (energy_reward + distance_penalty + success_reward + 
                       visited_penalty + energy_penalty + step_penalty + proximity_reward)
        
        return total_reward
    
    def _is_done(self) -> bool:
        """Vérifie si l'épisode est terminé (succès/échec)"""
        # Succès : arrivée à destination
        if self.state.current_node == self.destination:
            return True
        
        # Échec : nœud actuel mort
        if not self.nodes[self.state.current_node].is_alive():
            return True
        
        return False
    
    def _get_observation(self) -> np.ndarray:
        """Retourne l'observation actuelle"""
        if self.state is None:
            return np.zeros(self.observation_space.shape[0], dtype=np.float32)
        
        # Normalisation des valeurs
        obs = []
        
        # Nœud actuel et destination (normalisés)
        obs.append(self.state.current_node / self.num_nodes)
        obs.append(self.state.destination / self.num_nodes)
        obs.append(self.state.data_size / max(self.data_size_range))
        
        # Énergie des nœuds (normalisée)
        obs.extend(self.state.nodes_energy / 1000.0)
        
        # Position des nœuds (normalisée)
        pos = self.state.nodes_position
        obs.extend(pos.flatten() / 1000.0)
        
        # Température des nœuds (normalisée)
        obs.extend(self.state.nodes_temperature / 30.0)
        
        # Salinité des nœuds (normalisée)
        obs.extend(self.state.nodes_salinity / 40.0)
        
        # Masque des nœuds visités
        visited_mask = np.zeros(self.num_nodes)
        for node_id in self.state.visited_nodes:
            visited_mask[node_id] = 1.0
        obs.extend(visited_mask)
        
        # Longueur du chemin (normalisée)
        obs.append(len(self.state.path) / self.max_steps)
        
        return np.array(obs, dtype=np.float32)
    
    def _get_nodes_energy(self) -> np.ndarray:
        """Retourne l'énergie de tous les nœuds"""
        return np.array([node.energy for node in self.nodes], dtype=np.float32)
    
    def _get_nodes_position(self) -> np.ndarray:
        """Retourne la position de tous les nœuds"""
        positions = []
        for node in self.nodes:
            positions.append([node.x, node.y, node.z])
        return np.array(positions, dtype=np.float32)
    
    def _get_nodes_temperature(self) -> np.ndarray:
        """Retourne la température de tous les nœuds"""
        return np.array([node.temperature for node in self.nodes], dtype=np.float32)
    
    def _get_nodes_salinity(self) -> np.ndarray:
        """Retourne la salinité de tous les nœuds"""
        return np.array([node.salinity for node in self.nodes], dtype=np.float32)
    
    def _get_info(self) -> Dict[str, Any]:
        """Retourne les informations supplémentaires"""
        return {
            'step': self.step_count,
            'current_node': self.state.current_node,
            'destination': self.state.destination,
            'path_length': len(self.state.path),
            'visited_nodes': self.state.visited_nodes.copy(),
            'episode_stats': self.episode_stats.copy()
        }
    
    def render(self, mode: str = 'human') -> Optional[np.ndarray]:
        """Rendu de l'environnement"""
        if mode == 'human':
            print(f"Étape {self.step_count}: Nœud actuel {self.state.current_node}, "
                  f"Destination {self.state.destination}, "
                  f"Chemin: {self.state.path}")
        elif mode == 'rgb_array':
            # Pour la visualisation graphique
            return None
    
    def get_network_info(self) -> Dict[str, Any]:
        """Retourne les informations du réseau"""
        return {
            'num_nodes': self.num_nodes,
            'nodes': [
                {
                    'id': node.id,
                    'position': [node.x, node.y, node.z],
                    'energy': node.energy,
                    'temperature': node.temperature,
                    'salinity': node.salinity
                }
                for node in self.nodes
            ]
        }

print("✅ Environnement Gym UWSN créé")
